### __Defining an SMA Crossover Strategy__

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [4]:
# EUR/USD 가격 데이터를 불러온 뒤 data 데이터프레임에 저장
data = pd.read_csv('Course_Materials/eurusd.csv', parse_dates=['Date'], index_col='Date')
data

,price
Date,
2004-01-01,1.258194
2004-01-02,1.258194
2004-01-05,1.268698
2004-01-06,1.272103
2004-01-07,1.264095
...,...
2020-06-24,1.131235
2020-06-25,1.125087
2020-06-26,1.122208
